In [1]:
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [3]:
decision = pd.read_csv('dados_ciclo_basic/decision.csv.csv')
fifa_results = pd.read_csv('dados_ciclo_basic/FIFA Results.csv')
matches = pd.read_csv('dados_ciclo_basic/dados_copadomundo/matches_1930_2022.csv')
world_cup = pd.read_csv('dados_ciclo_basic/dados_copadomundo/world_cup.csv')